In [1]:
# Import libraries
import pandas as pd
import numpy as np
import itertools
from prophet import Prophet
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error, mean_absolute_error
from prophet.diagnostics import performance_metrics
from prophet.diagnostics import cross_validation
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.gridspec import GridSpec
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import month_plot, quarter_plot
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.utilities import regressor_coefficients
import utils
from prophet.plot import plot_cross_validation_metric
from prophet.plot import add_changepoints_to_plot
from prophet.serialize import model_to_json, model_from_json

In [2]:
def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
# Get Data
drv_data = pd.read_csv('/Users/didi/Documents/Short Term Forecast/Daily Folder/drv_data_ssl_full2022.csv',encoding='cp1252')
pax_data = pd.read_csv('/Users/didi/Documents/Short Term Forecast/Daily Folder/pax_data_ssl_full2022.csv',encoding='cp1252')
#eyeballs = pd.read_csv('/Users/didi/Documents/Short Term Forecast/eyeball_data_ssl_full2022.csv' ,encoding='cp1252')
burn = pd.read_csv('/Users/didi/Documents/Short Term Forecast/Daily Folder/burn_data_ssl_full2022.csv')
holidays = pd.read_csv('/Users/didi/Documents/Short Term Forecast/Daily Folder/SSL Central Ops - List of Holidays - holidays.csv')
paychecks = pd.read_csv('/Users/didi/Documents/Short Term Forecast/Daily Folder/ssl_paycheck_dates_full.csv')
dates = pd.read_csv('/Users/didi/Documents/Short Term Forecast/Daily Folder/date_data.csv')
burn['driver_income'].astype('float')
print(burn.info())
drv_data.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37938 entries, 0 to 37937
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   city_id             37938 non-null  int64  
 1   year_calendar_week  37938 non-null  object 
 2   current_stat_date   37938 non-null  object 
 3   country_code        37938 non-null  object 
 4   gmv                 37938 non-null  float64
 5   pax_eng_burn        37938 non-null  float64
 6   pax_aqc_burn        37938 non-null  float64
 7   drv_eng_burn        37938 non-null  float64
 8   drv_acq_burn        37938 non-null  float64
 9   driver_income       35356 non-null  float64
 10  driver_income_pct   35356 non-null  float64
 11  ipk                 35786 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 3.5+ MB
None


,current_stat_date,year_calendar_week,country_code,city_id,city_name,dfsh_cnt,dfsh_online_time,dfsh_shpd,retained_drv_cnt,retained_drv_online_time,retained_drv_shpd,active_drv_cnt,active_drv_online_time,active_drv_shpd
0,01/01/21,2021/01,CR,506070100,San Jos¨¦,5,21.916667,4.383333,3479,12932.71667,3.717366,3484,12954.63333,3.718322


In [4]:

exchange_rates = {
    'MX': 0.054,
    'AR': 0.0053,
    'PE': 0.26,
    'CR': 0.0018,
    'CO': 0.00021,
    'CL': 0.0013,
    'DO': 0.018,
    'PA': 1,
    'EC': 1
}
burn_poc = burn[(burn.city_id == 101)]
burn = burn[(burn.city_id != 101)]
#burn_poc[['gmv', 'pax_eng_burn', 'pax_aqc_burn', 'drv_eng_burn', 'drv_acq_burn', 'driver_income','ipk']] *= burn_poc['country_code'].map(exchange_rates)

def convert_to_usd(row):
    rate = exchange_rates[row['country_code']]
    row['gmv'] *= rate
    row['pax_eng_burn'] *= rate
    row['pax_aqc_burn'] *= rate
    row['drv_eng_burn'] *= rate
    row['drv_acq_burn'] *= rate
    row['driver_income'] *= rate
    row['ipk'] *= rate
    return row

burn_poc = burn_poc.apply(convert_to_usd, axis=1)

drv_data_poc = drv_data[drv_data.city_id == 101]
drv_data = drv_data[drv_data.city_id != 101]

pax_data_poc = pax_data[pax_data.city_id == 101]
pax_data = pax_data[pax_data.city_id != 101]

drv_data_poc.country_code[drv_data_poc.city_id == 101] = 'MX'
pax_data_poc.country_code[pax_data_poc.city_id == 101] = 'MX'
burn_poc.country_code[burn_poc.city_id == 101] = 'MX'
drv_data_poc = (
    drv_data_poc.groupby(['current_stat_date', 'year_calendar_week', 'country_code', 'city_id', 'city_name'])
    .agg({
    'dfsh_cnt':'sum',
    'dfsh_online_time':'sum',
    'dfsh_shpd':'mean',
    'retained_drv_cnt':'sum',
    'retained_drv_online_time':'sum',
    'retained_drv_shpd':'mean',
    'active_drv_cnt':'sum',
    'active_drv_online_time':'sum',
    'active_drv_shpd':'mean'
    })
    .reset_index()
    )

pax_data_poc = (
    pax_data_poc.groupby(['current_stat_date', 'year_calendar_week', 'country_code', 'city_name','city_id'])
    .agg({
    'pfc_pax_cnt':'sum',
    'pfc_calls':'sum',
    'pfc_cpp':'mean',
    'retained_pax_cnt':'sum',
    'retained_calls':'sum',
    'retained_cpp':'mean',
    'active_pax_cnt':'sum',
    'active_pax_calls':'sum',
    'active_pax_cpp':'mean',
    'rides':'sum' 
    })
    .reset_index()
    )
    
burn_poc = (
    burn_poc.groupby(['city_id', 'year_calendar_week', 'current_stat_date', 'country_code'])
    .agg({
    'gmv':'sum',
    'pax_eng_burn':'sum',
    'pax_aqc_burn':'mean',
    'drv_eng_burn':'sum',
    'drv_acq_burn':'sum',
    'driver_income':'mean',
    'ipk': 'mean',
    'driver_income_pct':'mean'
   
    })
    .reset_index()
    )

drv_data_poc['dfsh_shpd'] = drv_data_poc.dfsh_online_time / drv_data_poc.dfsh_cnt
drv_data_poc['retained_drv_shpd'] = drv_data_poc.retained_drv_online_time / drv_data_poc.retained_drv_cnt

pax_data_poc['pfc_cpp'] = pax_data_poc.pfc_calls / pax_data_poc.pfc_pax_cnt
pax_data_poc['retained_cpp'] =  pax_data_poc.retained_calls / pax_data_poc.retained_pax_cnt

burn_poc['driver_income_pct'] = burn_poc.driver_income / burn_poc.gmv

drv_data = pd.concat([drv_data, drv_data_poc])
pax_data = pd.concat([pax_data, pax_data_poc])
burn = pd.concat([burn, burn_poc])

/var/folders/qq/jd7pmms11gv5jnzmr9cwqgk40000gn/T/ipykernel_9677/358684461.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  burn_poc.country_code[burn_poc.city_id == 101] = 'MX'


In [5]:
# clean burn data
burn = burn[burn['city_id'].notnull()]
burn.city_id = burn.city_id.apply(np.int64)
burn.pax_aqc_burn = np.where(burn.pax_aqc_burn.isnull(),0,burn.pax_aqc_burn)
burn.drv_acq_burn = np.where(burn.drv_acq_burn.isnull(),0,burn.drv_acq_burn)
burn.pax_eng_burn = np.where(burn.pax_eng_burn.isnull(),0,burn.pax_eng_burn)
burn.drv_eng_burn = np.where(burn.drv_eng_burn.isnull(),0,burn.drv_eng_burn)
burn = burn[burn.city_id != -99999]
burn= burn.assign(pax_acq_burn_pct = burn.pax_aqc_burn/ burn.gmv
                  ,drv_acq_burn_pct = burn.drv_acq_burn/ burn.gmv
                  ,pax_eng_burn_pct = burn.pax_eng_burn/ burn.gmv
                  ,drv_eng_burn_pct = burn.drv_eng_burn/ burn.gmv)

burn.pax_acq_burn_pct = np.where(burn.pax_acq_burn_pct.isnull(),0,burn.pax_acq_burn_pct)
burn.drv_acq_burn_pct = np.where(burn.drv_acq_burn_pct.isnull(),0,burn.drv_acq_burn_pct)
burn.pax_eng_burn_pct = np.where(burn.pax_eng_burn_pct.isnull(),0,burn.pax_eng_burn_pct)
burn.drv_eng_burn_pct = np.where(burn.drv_eng_burn_pct.isnull(),0,burn.drv_eng_burn_pct)

# Merge pax and driver data
df1 = (pax_data
        .merge(drv_data, how='left', on = ['current_stat_date','city_id','year_calendar_week', 'country_code'])
        .merge(burn, how='left', on = ['current_stat_date','city_id', 'country_code','year_calendar_week'])
        .drop(columns=['city_name_y'])
        .rename(columns = {'city_name_x':'city_name'})
    )
# Transfrom to Datetime Format
df1.current_stat_date = pd.to_datetime(df1.current_stat_date, format = '%d/%m/%y')
df1.head(1)
print(df1.columns)
# Transfrom to Datetime Format and drop columns
paychecks.date = pd.to_datetime(paychecks.date, format= '%Y-%m-%d')
paychecks.drop(columns = ['year_calendar_week', 'paycheck'], inplace = True)
dates.date_value = pd.to_datetime(dates.date_value,  format='%d/%m/%y')

# Final DataFrame with paycheck
master = (df1.merge(paychecks, how = 'left', left_on = ['current_stat_date', 'country_code'], right_on = ['date', 'country_code'])
      .drop(columns = ['year_calendar_week', 'city_name', 'date'])
      .sort_values('current_stat_date', ascending = True))



master['day_of_week'] = master.current_stat_date.dt.day_name()
col = ['day_of_week' ]
for i in col: 
      master[i] = master[i].astype('category')
master =  pd.concat((master,pd.get_dummies(master['day_of_week'], drop_first= True)), axis = 1)

date_pred =  pd.DataFrame(dates, columns = ['date_value', 'rank', 'weekday'])
date_pred['day_of_week'] = date_pred.date_value.dt.day_name()
col = ['day_of_week']
for i in col: 
      date_pred[i] = date_pred[i].astype('category')
date_pred =  pd.concat((date_pred,pd.get_dummies(date_pred['day_of_week'], drop_first= True)), axis = 1)

master['ipt'] = master['driver_income'] / master['rides']
master['ipt_standardized'] = master.groupby('country_code')['ipt'].apply(lambda x: (x - x.mean()) / x.std())

master = master.replace([np.inf, -np.inf], 0)
master = master.fillna(0)

# Remove Data errors

mapped_cities = {'country_code': ['MX','MX','MX','MX','MX','MX','MX','MX','MX','MX','MX','MX','MX','MX','MX','MX','MX','MX','MX','DO','EC','CO','CR','CL','CO','DO','CL','AR','EC','CO','MX'],
                 'city_id': [52090100,52190500,52140500,52210400,52151600,52310300,52220400,52020200,52020100,52080800,52110500,52170200,52080200,52250100,52010100,52260200,52280300,52250300,52050400,809230100,593071200,57330100,506070100,56513800,57380100,809190100,56650800,54052300,593080100,57010100,101]}

mapped_cities = pd.DataFrame.from_dict(mapped_cities)
master = master.merge(mapped_cities, how = 'inner', on = ['country_code', 'city_id'])

master.to_csv('ssl_full_procssed_data.csv', index = False)

/var/folders/qq/jd7pmms11gv5jnzmr9cwqgk40000gn/T/ipykernel_9677/3944765788.py:56: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  master['ipt_standardized'] = master.groupby('country_code')['ipt'].apply(lambda x: (x - x.mean()) / x.std())


Index(['current_stat_date', 'year_calendar_week', 'country_code', 'city_name',
       'city_id', 'pfc_pax_cnt', 'pfc_calls', 'pfc_cpp', 'retained_pax_cnt',
       'retained_calls', 'retained_cpp', 'active_pax_cnt', 'active_pax_calls',
       'active_pax_cpp', 'rides', 'dfsh_cnt', 'dfsh_online_time', 'dfsh_shpd',
       'retained_drv_cnt', 'retained_drv_online_time', 'retained_drv_shpd',
       'active_drv_cnt', 'active_drv_online_time', 'active_drv_shpd', 'gmv',
       'pax_eng_burn', 'pax_aqc_burn', 'drv_eng_burn', 'drv_acq_burn',
       'driver_income', 'driver_income_pct', 'ipk', 'pax_acq_burn_pct',
       'drv_acq_burn_pct', 'pax_eng_burn_pct', 'drv_eng_burn_pct'],
      dtype='object')


In [6]:
# Initialize parameter

 # Test Days
testdays = 30

# Train limit
master = master[master.current_stat_date <= '2023-01-25']

# Horizon for MAPE
horizon = "30 days"

# Forecast Days after training specified date
forecast = 36

# Output boundaries
start_date = "2022-06-01"
end_date = "2023-03-27"

# Cities to model
city_list = [           
52090100
,52190500
,52140500
,52210400
,52151600
,52310300
,52220400
,52020200
,52020100
,52080800
,52110500
,52170200
,52080200
,52250100
,52010100
,52260200
,52280300
,52250300
,52050400
,809230100
#,593071200 # Guayaquil
,57330100
,506070100
,56513800
,57380100
#,809190100 #Santiago
,56650800
,54052300
,593080100
,57010100
#,101 # POC
] # --> CDMX  52090100


In [7]:
city_df= pd.DataFrame(drv_data, columns=['city_id', 'country_code'])
city_df = city_df.drop_duplicates()
city_df = city_df[city_df.city_id.isin(city_list)]
city_country = pd.Series(city_df.country_code.values, index=city_df.city_id).to_dict()
city_country

{506070100: 'CR',
 52210400: 'MX',
 52151600: 'MX',
 52110500: 'MX',
 52140500: 'MX',
 52190500: 'MX',
 52020200: 'MX',
 52080800: 'MX',
 52250300: 'MX',
 809230100: 'DO',
 54052300: 'AR',
 52020100: 'MX',
 57010100: 'CO',
 57330100: 'CO',
 52080200: 'MX',
 56650800: 'CL',
 56513800: 'CL',
 52010100: 'MX',
 52220400: 'MX',
 52280300: 'MX',
 52090100: 'MX',
 52250100: 'MX',
 57380100: 'CO',
 593080100: 'EC',
 52170200: 'MX',
 52050400: 'MX',
 52260200: 'MX',
 52310300: 'MX'}

In [ ]:
market = []

for i,j in city_country.items():

    # Mater File of Predicted PAX Variables
    master_pfc_df = (pd.DataFrame(
        data = master, columns=['current_stat_date', 'pfc_pax_cnt', 'paycheck_day', 'city_id','pax_acq_burn_pct', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday'])
        .rename(columns = {'current_stat_date': 'ds','pfc_pax_cnt': 'y' }))
    master_pfc_df.pax_acq_burn_pct = np.where(master_pfc_df.pax_acq_burn_pct.isnull(),0,master_pfc_df.pax_acq_burn_pct)
    master_pfc_df = master_pfc_df[(master_pfc_df.city_id ==i)].drop(columns = 'city_id')

    master_pax_df = (pd.DataFrame(
        data = master, columns=['current_stat_date', 'retained_pax_cnt', 'paycheck_day', 'city_id','pax_eng_burn_pct', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday'])
        .rename(columns = {'current_stat_date': 'ds','retained_pax_cnt': 'y' }))
    master_pax_df.pax_eng_burn_pct = np.where(master_pax_df.pax_eng_burn_pct.isnull(),0,master_pax_df.pax_eng_burn_pct)
    master_pax_df = master_pax_df[(master_pax_df.city_id == i)].drop(columns = 'city_id')

    # Mater File of Predicted DRV Variables
    master_dfsh_df = (pd.DataFrame(
        data = master, columns=['current_stat_date', 'dfsh_cnt', 'paycheck_day', 'city_id','drv_acq_burn_pct', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday'])
        .rename(columns = {'current_stat_date': 'ds','dfsh_cnt': 'y' }))
    master_dfsh_df.drv_acq_burn_pct = np.where(master_dfsh_df.drv_acq_burn_pct.isnull(),0,master_dfsh_df.drv_acq_burn_pct)
    master_dfsh_df = master_dfsh_df[(master_dfsh_df.city_id ==i)].drop(columns = 'city_id')

    master_drv_df = (pd.DataFrame(
        data = master, columns=['current_stat_date', 'retained_drv_cnt', 'paycheck_day', 'city_id','drv_eng_burn_pct','ipt_standardized', 'ipt','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday'])
        .rename(columns = {'current_stat_date': 'ds','retained_drv_cnt': 'y' }))
    #master_drv_df.drv_eng_burn_pct = np.where(master_drv_df.drv_eng_burn_pct.isnull(),0,master_drv_df.drv_eng_burn_pct)
    master_drv_df = master_drv_df[(master_drv_df.city_id ==i)].drop(columns = 'city_id')

    # Master Holidays File
    master_holidays = holidays[holidays.country_code == j]

    # Holidays Modelling
    master_holidays.date_value = pd.to_datetime(master_holidays.date_value, format='%Y-%m-%d')
    master_holidays = master_holidays.drop(columns = ['new_year_calendar_week', 'subregion', 'year', 'month', 'day', 'date_id', 'date',
        'weekday'])
    # Holiday Dataframe
    master_holidays = pd.DataFrame({'holiday': master_holidays.holiday_name, 'ds': master_holidays.date_value, 'lower_window': -1, 'upper_window':1})
    

    # Slicing Datasets for PAX
    pfc_training_set = master_pfc_df.iloc[:-testdays,:]
    pfc_test_set = master_pfc_df.iloc[-testdays:,:]

    pax_training_set = master_pax_df.iloc[:-testdays,:]
    pax_test_set = master_pax_df.iloc[-testdays:,:]

    # Slicing Datasets for DRV
    dfsh_training_set = master_dfsh_df.iloc[:-testdays,:]
    dfsh_test_set = master_dfsh_df.iloc[-testdays:,:]

    drv_training_set = master_drv_df.iloc[:-testdays,:]
    drv_test_set = master_drv_df.iloc[-testdays:,:]

    # Mater File of Predicted PAX Variables
    pfc_freq_df = (pd.DataFrame(
        data = master, columns=['current_stat_date','city_id', 'pfc_cpp','pfc_pax_cnt','pfc_calls'])
        .rename(columns = {'current_stat_date': 'ds'}))
    pfc_freq_df = pfc_freq_df[(pfc_freq_df.city_id == i)]

    pax_freq_df = (pd.DataFrame(
        data = master, columns=['current_stat_date','city_id','retained_cpp','retained_pax_cnt','retained_calls' ])
        .rename(columns = {'current_stat_date': 'ds'}))
    pax_freq_df = pax_freq_df[(pax_freq_df.city_id == i)]

    # Mater File of Predicted DRV Variables
    dfsh_freq_df = (pd.DataFrame(
        data = master, columns=['current_stat_date','city_id','dfsh_shpd','dfsh_cnt', 'dfsh_online_time'])
        .rename(columns = {'current_stat_date': 'ds'}))
    dfsh_freq_df = dfsh_freq_df[(dfsh_freq_df.city_id ==i)]

    drv_freq_df = (pd.DataFrame(
        data = master, columns=['current_stat_date','city_id', 'retained_drv_shpd','retained_drv_cnt', 'retained_drv_online_time'])
        .rename(columns = {'current_stat_date': 'ds'}))
    drv_freq_df = drv_freq_df[(drv_freq_df.city_id == i)]

    # Creating frequency datasets with dates
    columns_drop = ['date_id','month','month_name','day','day_of_year','weekday_name','formatted_date','quartal','year_quartal','year_month', 'weekend', 'period','ds']

    pfc_freq_df = dates.merge(pfc_freq_df, how = 'left', left_on = 'date_value', right_on = 'ds').drop(columns=columns_drop)
    pfc_freq_df = pfc_freq_df[(pfc_freq_df.date_value >= '2020-09-01')&(pfc_freq_df.date_value <= '2023-12-31')]

    pax_freq_df = dates.merge(pax_freq_df, how = 'left', left_on = 'date_value', right_on = 'ds').drop(columns=columns_drop)
    pax_freq_df = pax_freq_df[(pax_freq_df.date_value >= '2020-09-01')&(pax_freq_df.date_value <= '2023-12-31')]

    dfsh_freq_df = dates.merge(dfsh_freq_df, how = 'left', left_on = 'date_value', right_on = 'ds').drop(columns=columns_drop)
    dfsh_freq_df = dfsh_freq_df[(dfsh_freq_df.date_value >= '2020-09-01')&(dfsh_freq_df.date_value <= '2023-12-31')]

    drv_freq_df = dates.merge(drv_freq_df, how = 'left', left_on = 'date_value', right_on = 'ds').drop(columns=columns_drop)
    drv_freq_df = drv_freq_df[(drv_freq_df.date_value >= '2020-09-01')&(drv_freq_df.date_value <= '2023-12-31')]

    # Left join with prev year and weeks

    # PFC frequency DF Joins
    pfc_freq_df['last_year'] = pfc_freq_df['rank'] +52

    pfc_freq_df['last_week'] = pfc_freq_df['rank'] +1
    pfc_freq_df['last2_week'] = pfc_freq_df['rank'] +2
    pfc_freq_df['last3_week'] = pfc_freq_df['rank'] +3
    pfc_freq_df['last4_week'] = pfc_freq_df['rank'] +4

    pfc_freq_final = (
        pfc_freq_df.merge(pfc_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = ['last_year','weekday'],suffixes=( None, '_ly'))
        .merge(pfc_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = [ 'last_week', 'weekday'],suffixes=( None, '_lw'))
        .merge(pfc_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = [ 'last2_week', 'weekday'],suffixes=( None, '_l2w'))
        .merge(pfc_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = [ 'last3_week', 'weekday'],suffixes=( None, '_l3w'))
        .merge(pfc_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = ['last4_week', 'weekday'],suffixes=( None, '_l4w'))
        )
    pfc_freq_final = pd.DataFrame(pfc_freq_final, columns=['rank','year','year_calendar_week','calendar_week','date_value','weekday','city_id', 'pfc_cpp', 'pfc_cpp_ly', 'pfc_cpp_lw', 'pfc_cpp_l2w', 'pfc_cpp_l3w', 'pfc_cpp_l4w'])

    # Pax frequency DF Joins
    pax_freq_df['last_year'] = pax_freq_df['rank']+52

    pax_freq_df['last_week'] = pax_freq_df['rank'] +1
    pax_freq_df['last2_week'] = pax_freq_df['rank'] +2
    pax_freq_df['last3_week'] = pax_freq_df['rank'] +3
    pax_freq_df['last4_week'] = pax_freq_df['rank'] +4

    pax_freq_final = (
        pax_freq_df.merge(pax_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = ['last_year', 'weekday'],suffixes=( None, '_ly'))
        .merge(pax_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = ['last_week', 'weekday'],suffixes=( None, '_lw'))
        .merge(pax_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = ['last2_week', 'weekday'],suffixes=( None, '_l2w'))
        .merge(pax_freq_df, how = 'left', left_on = ['rank', 'weekday'], right_on = ['last3_week', 'weekday'],suffixes=( None, '_l3w'))
        .merge(pax_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = ['last4_week', 'weekday'],suffixes=( None, '_l4w'))
        )
    pax_freq_final = pd.DataFrame(pax_freq_final, columns=['rank','year','year_calendar_week','calendar_week','date_value','weekday','city_id', 'retained_cpp', 'retained_cpp_ly', 'retained_cpp_lw','retained_cpp_l2w', 'retained_cpp_l3w','retained_cpp_l4w'])

    # DFSH frequency DF Joins
    dfsh_freq_df['last_year'] = dfsh_freq_df['rank'] +52

    dfsh_freq_df['last_week'] = dfsh_freq_df['rank'] +1
    dfsh_freq_df['last2_week'] = dfsh_freq_df['rank'] +2
    dfsh_freq_df['last3_week'] = dfsh_freq_df['rank'] +3
    dfsh_freq_df['last4_week'] = dfsh_freq_df['rank'] +4

    dfsh_freq_final = (
        dfsh_freq_df.merge(dfsh_freq_df, how = 'left', left_on = ['rank', 'weekday'], right_on = ['last_year', 'weekday'],suffixes=( None, '_ly'))
        .merge(dfsh_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = ['last_week', 'weekday'],suffixes=( None, '_lw'))
        .merge(dfsh_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = ['last2_week', 'weekday'],suffixes=( None, '_l2w'))
        .merge(dfsh_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = ['last3_week', 'weekday'],suffixes=( None, '_l3w'))
        .merge(dfsh_freq_df, how = 'left', left_on = [ 'rank', 'weekday'], right_on = ['last4_week', 'weekday'],suffixes=( None, '_l4w'))
        )
    dfsh_freq_final = pd.DataFrame(dfsh_freq_final, columns=['rank','year','year_calendar_week','calendar_week','date_value','weekday','city_id', 'dfsh_shpd', 'dfsh_shpd_ly', 'dfsh_shpd_lw','dfsh_shpd_l2w', 'dfsh_shpd_l3w','dfsh_shpd_l4w'])

    # DRV frequency DF Joins
    drv_freq_df['last_year'] = drv_freq_df['rank'] +52

    drv_freq_df['last_week'] = drv_freq_df['rank'] +1
    drv_freq_df['last2_week'] = drv_freq_df['rank'] +2
    drv_freq_df['last3_week'] = drv_freq_df['rank'] +3
    drv_freq_df['last4_week'] = drv_freq_df['rank'] +4

    drv_freq_final = (
        drv_freq_df.merge(drv_freq_df, how = 'left', left_on = ['rank', 'weekday'], right_on = ['last_year', 'weekday'],suffixes=( None, '_ly'))
        .merge(drv_freq_df, how = 'left', left_on = ['rank',  'weekday'], right_on = ['last_week', 'weekday'],suffixes=( None, '_lw'))
        .merge(drv_freq_df, how = 'left', left_on = ['rank',  'weekday'], right_on = ['last2_week', 'weekday'],suffixes=( None, '_l2w'))
        .merge(drv_freq_df, how = 'left', left_on = ['rank',  'weekday'], right_on = ['last3_week', 'weekday'],suffixes=( None, '_l3w'))
        .merge(drv_freq_df, how = 'left', left_on = ['rank',  'weekday'], right_on = ['last4_week', 'weekday'],suffixes=( None, '_l4w'))
        )
    drv_freq_final = pd.DataFrame(drv_freq_final
                    , columns=['rank','year','year_calendar_week','calendar_week','date_value','weekday','city_id', 'retained_drv_shpd', 'retained_drv_shpd_ly', 'retained_drv_shpd_lw','retained_drv_shpd_l2w'
                    , 'retained_drv_shpd_l3w','retained_drv_shpd_l4w','last_week', 'last_2week', 'last_3week', 'last_4week' ])

    pfc_freq_final['pfc_cpp_lweeks'] = (pfc_freq_final.pfc_cpp_lw
                                    + pfc_freq_final.pfc_cpp_l2w
                                    + pfc_freq_final.pfc_cpp_l3w
                                    + pfc_freq_final.pfc_cpp_l4w ) / 4


    pax_freq_final['retained_cpp_lweeks'] = (pax_freq_final.retained_cpp_lw
                                    + pax_freq_final.retained_cpp_l2w
                                    + pax_freq_final.retained_cpp_l3w
                                    + pax_freq_final.retained_cpp_l4w ) / 4

    dfsh_freq_final['dfsh_shpd_lweeks'] = (dfsh_freq_final.dfsh_shpd_lw
                                    + dfsh_freq_final.dfsh_shpd_l2w
                                    + dfsh_freq_final.dfsh_shpd_l3w
                                    + dfsh_freq_final.dfsh_shpd_l4w ) / 4

    drv_freq_final['retained_drv_shpd_lweeks'] = (drv_freq_final.retained_drv_shpd_lw
                                    + drv_freq_final.retained_drv_shpd_l2w
                                    + drv_freq_final.retained_drv_shpd_l3w
                                    + drv_freq_final.retained_drv_shpd_l4w ) / 4

    pfc_freq_final = pd.DataFrame(pfc_freq_final, columns = ['rank', 'year', 'year_calendar_week', 'calendar_week', 'date_value', 'weekday', 'city_id', 'pfc_cpp_ly', 'pfc_cpp_lweeks', 'pfc_cpp'])
    pfc_freq_final['pfc_cpp_lweeks'].ffill(axis=0,inplace=True)
    pfc_freq_final['city_id'].ffill(axis=0,inplace=True)
    pfc_freq_final['pfc_cpp_pred'] = (pfc_freq_final.pfc_cpp_lweeks*0.2+pfc_freq_final.pfc_cpp_ly*0.8)
    pfc_freq_final['pfc_cpp_pred'] =np.where(pfc_freq_final.pfc_cpp_pred.isnull(),pfc_freq_final.pfc_cpp,pfc_freq_final.pfc_cpp_pred)

    pax_freq_final = pd.DataFrame(pax_freq_final, columns = ['rank', 'year', 'year_calendar_week', 'calendar_week', 'date_value', 'weekday', 'city_id', 'retained_cpp_ly', 'retained_cpp_lweeks','retained_cpp' ])
    pax_freq_final['retained_cpp_lweeks'].ffill(axis=0,inplace=True)
    pax_freq_final['city_id'].ffill(axis=0,inplace=True)
    pax_freq_final['retained_cpp_pred'] = (pax_freq_final.retained_cpp_lweeks*0.3+pax_freq_final.retained_cpp_ly*0.7)
    pax_freq_final['retained_cpp_pred'] = np.where(pax_freq_final.retained_cpp_pred.isnull(),pax_freq_final.retained_cpp,pax_freq_final.retained_cpp_pred)

    dfsh_freq_final = pd.DataFrame(dfsh_freq_final, columns = ['rank', 'year', 'year_calendar_week', 'calendar_week', 'date_value', 'weekday', 'city_id', 'dfsh_shpd_ly', 'dfsh_shpd_lweeks','dfsh_shpd'])
    dfsh_freq_final['dfsh_shpd_lweeks'].ffill(axis=0,inplace=True)
    dfsh_freq_final['city_id'].ffill(axis=0,inplace=True)
    dfsh_freq_final['dfsh_shpd_pred'] = (dfsh_freq_final.dfsh_shpd_lweeks*0.4+dfsh_freq_final.dfsh_shpd_ly*0.6)
    dfsh_freq_final['dfsh_shpd_pred'] = np.where(dfsh_freq_final.dfsh_shpd_pred.isnull(),dfsh_freq_final.dfsh_shpd, dfsh_freq_final.dfsh_shpd_pred)

    drv_freq_final = pd.DataFrame(drv_freq_final, columns = ['rank', 'year', 'year_calendar_week', 'calendar_week', 'date_value', 'weekday', 'city_id', 'retained_drv_shpd_ly', 'retained_drv_shpd_lweeks','retained_drv_shpd'])
    drv_freq_final['retained_drv_shpd_lweeks'].ffill(axis=0,inplace=True)
    drv_freq_final['city_id'].ffill(axis=0,inplace=True)
    drv_freq_final['retained_drv_shpd_pred'] = (drv_freq_final.retained_drv_shpd_lweeks*0.5+drv_freq_final.retained_drv_shpd_ly*0.5)
    drv_freq_final['retained_drv_shpd_pred'] = np.where(drv_freq_final.retained_drv_shpd_pred.isnull(),drv_freq_final.retained_drv_shpd,drv_freq_final.retained_drv_shpd_pred)

    frequency_df = (pfc_freq_final.merge(pax_freq_final, how = 'left', on = 'date_value', suffixes = (None, '_x'))
                    .merge(dfsh_freq_final, how = 'left', on = 'date_value', suffixes = (None, '_y'))
                    .merge(drv_freq_final, how = 'left', on = 'date_value', suffixes = (None, '_z'))
                    )
    frequency_df = pd.DataFrame(frequency_df, columns = ['rank', 'year', 'year_calendar_week', 'calendar_week', 'date_value', 'weekday', 'city_id','pfc_cpp_pred','retained_cpp_pred','dfsh_shpd_pred','retained_drv_shpd_pred','pfc_cpp','retained_cpp','dfsh_shpd','retained_drv_shpd'])
    
    # Parameter Grid
    param_grid = {
        'seasonality_prior_scale':   [0.01,0.5,5]
        , 'changepoint_prior_scale':  [0.001,0.05,0.1]
        , 'holidays_prior_scale': [0.01,0.5,5]
        , 'seasonality_mode': [ 'multiplicative'] # 'additive',
        }
    grid = ParameterGrid(param_grid)

    # Hyperparameter tuning PFC
    pfc_mape = []
    for params in grid: 
        #build model
        pfc_m = (Prophet(
            growth = 'linear'
            , yearly_seasonality = True
            , weekly_seasonality = True
            , daily_seasonality = True
            , seasonality_mode = params['seasonality_mode']
            , holidays = master_holidays
            , seasonality_prior_scale = params['seasonality_prior_scale']
            , holidays_prior_scale = params['holidays_prior_scale']
            , changepoint_prior_scale=  params['changepoint_prior_scale']
            )
        )
        pfc_m.add_regressor('pax_acq_burn_pct')
        pfc_m.add_regressor('paycheck_day')
        pfc_m.add_regressor('Monday')
        pfc_m.add_regressor('Tuesday')
        pfc_m.add_regressor('Wednesday')
        pfc_m.add_regressor('Thursday')
        pfc_m.add_regressor('Saturday')
        pfc_m.add_regressor('Sunday')
        pfc_m.fit(pfc_training_set)
        #cross-validation
        pfc_cv = cross_validation(pfc_m, horizon = horizon, initial = '366 days', parallel= 'processes')
        #gather results
        pfc_error = MAPE(pfc_cv['y'], pfc_cv['yhat'])
        
        pfc_mape.append(pfc_error)
    # Hyperparameter tuning Retained Pax
    pax_mape = []
    for params in grid: 
        #build model
        pax_m = (Prophet(
            growth = 'linear'
            , yearly_seasonality = True
            , weekly_seasonality = True
            , daily_seasonality = True
            , seasonality_mode = params['seasonality_mode']
            , holidays = master_holidays
            , seasonality_prior_scale = params['seasonality_prior_scale']
            , holidays_prior_scale = params['holidays_prior_scale']
            , changepoint_prior_scale=  params['changepoint_prior_scale']
            )
        )
        pax_m.add_regressor('pax_eng_burn_pct')
        pax_m.add_regressor('paycheck_day')
        pax_m.add_regressor('Monday')
        pax_m.add_regressor('Tuesday')
        pax_m.add_regressor('Wednesday')
        pax_m.add_regressor('Thursday')
        pax_m.add_regressor('Saturday')
        pax_m.add_regressor('Sunday')
        pax_m.fit(pax_training_set)
        #cross-validation
        pax_cv = cross_validation(pax_m, horizon = horizon, initial = '366 days', parallel= 'processes')
        #gather results
        pax_error = MAPE(pax_cv['y'], pax_cv['yhat'])
        
        pax_mape.append(pax_error)
        
    # Hyperparameter tuning DFSH
    dfsh_mape = []
    for params in grid: 
        #build model
        dfsh_m = (Prophet(
            growth = 'linear'
            , yearly_seasonality = True
            , weekly_seasonality = True
            , daily_seasonality = True
            , seasonality_mode = params['seasonality_mode']
            , holidays = master_holidays
            , seasonality_prior_scale = params['seasonality_prior_scale']
            , holidays_prior_scale = params['holidays_prior_scale']
            , changepoint_prior_scale=  params['changepoint_prior_scale']
            )
        )
        dfsh_m.add_regressor('drv_acq_burn_pct')
        dfsh_m.add_regressor('paycheck_day')
        dfsh_m.add_regressor('Monday')
        dfsh_m.add_regressor('Tuesday')
        dfsh_m.add_regressor('Wednesday')
        dfsh_m.add_regressor('Thursday')
        dfsh_m.add_regressor('Saturday')
        dfsh_m.add_regressor('Sunday')
        dfsh_m.fit(dfsh_training_set)
        #cross-validation
        dfsh_cv = cross_validation(dfsh_m, horizon = horizon, initial = '366 days', parallel= 'processes')
        #gather results
        dfsh_error = MAPE(dfsh_cv['y'], dfsh_cv['yhat'])
        
        dfsh_mape.append(dfsh_error)
        
    # Hyperparameter tuning Retained Drv
    drv_mape = []
    for params in grid: 
        #build model
        drv_m = (Prophet(
            growth = 'linear'
            , yearly_seasonality = True
            , weekly_seasonality = True
            , daily_seasonality = True
            , seasonality_mode = params['seasonality_mode']
            , holidays = master_holidays
            , seasonality_prior_scale = params['seasonality_prior_scale']
            , holidays_prior_scale = params['holidays_prior_scale']
            , changepoint_prior_scale=  params['changepoint_prior_scale']
            )
        )
        drv_m.add_regressor('ipt_standardized')
        drv_m.add_regressor('drv_eng_burn_pct')
        drv_m.add_regressor('paycheck_day')
        drv_m.add_regressor('Monday')
        drv_m.add_regressor('Tuesday')
        drv_m.add_regressor('Wednesday')
        drv_m.add_regressor('Thursday')
        drv_m.add_regressor('Saturday')
        drv_m.add_regressor('Sunday')
        drv_m.fit(drv_training_set)
        #cross-validation
        drv_cv = cross_validation(drv_m, horizon = horizon, initial = '366 days', parallel= 'processes')
        #gather results
        drv_error = MAPE(drv_cv['y'], drv_cv['yhat'])
        
        drv_mape.append(drv_error)

    # best parameters pax
    pfc_best_params = grid[np.argmin(pfc_mape)]

    pax_best_params = grid[np.argmin(pax_mape)]

    # best parameters drv
    dfsh_best_params = grid[np.argmin(dfsh_mape)]

    drv_best_params = grid[np.argmin(drv_mape)]

    # PAX Final models
    pfc_m = Prophet(growth = 'linear'
                , yearly_seasonality = True
                , weekly_seasonality = True
                , daily_seasonality = True
                , seasonality_mode = pfc_best_params['seasonality_mode']
                , seasonality_prior_scale = pfc_best_params['seasonality_prior_scale']
                , holidays_prior_scale = pfc_best_params['holidays_prior_scale']
                , holidays = master_holidays
                , changepoint_prior_scale = pfc_best_params['changepoint_prior_scale']
    )
    pfc_m.add_regressor('pax_acq_burn_pct')
    pfc_m.add_regressor('paycheck_day')
    pfc_m.add_regressor('Monday')
    pfc_m.add_regressor('Tuesday')
    pfc_m.add_regressor('Wednesday')
    pfc_m.add_regressor('Thursday')
    pfc_m.add_regressor('Saturday')
    pfc_m.add_regressor('Sunday')
    pfc_m.fit(pfc_training_set)

    pax_m = Prophet(growth = 'linear'
                , yearly_seasonality = True
                , weekly_seasonality = True
                , daily_seasonality = True
                , seasonality_mode = pax_best_params['seasonality_mode']
                , seasonality_prior_scale = pax_best_params['seasonality_prior_scale']
                , holidays_prior_scale = pax_best_params['holidays_prior_scale']
                , holidays = master_holidays
                , changepoint_prior_scale = pax_best_params['changepoint_prior_scale']
    )
    pax_m.add_regressor('pax_eng_burn_pct')
    pax_m.add_regressor('paycheck_day')
    pax_m.add_regressor('Monday')
    pax_m.add_regressor('Tuesday')
    pax_m.add_regressor('Wednesday')
    pax_m.add_regressor('Thursday')
    pax_m.add_regressor('Saturday')
    pax_m.add_regressor('Sunday')
    pax_m.fit(pax_training_set)

    # DRV Final models
    dfsh_m = Prophet(growth = 'linear'
                , yearly_seasonality = True
                , weekly_seasonality = True
                , daily_seasonality = True
                , seasonality_mode = dfsh_best_params['seasonality_mode']
                , seasonality_prior_scale = dfsh_best_params['seasonality_prior_scale']
                , holidays_prior_scale = dfsh_best_params['holidays_prior_scale']
                , holidays = master_holidays
                , changepoint_prior_scale = dfsh_best_params['changepoint_prior_scale']
    )
    dfsh_m.add_regressor('drv_acq_burn_pct')
    dfsh_m.add_regressor('paycheck_day')
    dfsh_m.add_regressor('Monday')
    dfsh_m.add_regressor('Tuesday')
    dfsh_m.add_regressor('Wednesday')
    dfsh_m.add_regressor('Thursday')
    dfsh_m.add_regressor('Saturday')
    dfsh_m.add_regressor('Sunday')
    dfsh_m.fit(dfsh_training_set)

    drv_m = Prophet(growth = 'linear'
                , yearly_seasonality = True
                , weekly_seasonality = True
                , daily_seasonality = True
                , seasonality_mode = drv_best_params['seasonality_mode']
                , seasonality_prior_scale = drv_best_params['seasonality_prior_scale']
                , holidays_prior_scale =  drv_best_params['holidays_prior_scale']
                , holidays = master_holidays
                , changepoint_prior_scale = drv_best_params['changepoint_prior_scale']
    )
    drv_m.add_regressor('ipt_standardized')
    drv_m.add_regressor('drv_eng_burn_pct')
    drv_m.add_regressor('paycheck_day')
    drv_m.add_regressor('Monday')
    drv_m.add_regressor('Tuesday')
    drv_m.add_regressor('Wednesday')
    drv_m.add_regressor('Thursday')
    drv_m.add_regressor('Saturday')
    drv_m.add_regressor('Sunday')
    drv_m.fit(drv_training_set)
    

    # with open('pfc_model_mx'+i+'.json', 'w') as fout:
    #     fout.write(model_to_json(pfc_m))  # Save model
    
    # with open('pax_model_mx'+i+'.json', 'w') as fout:
    #     fout.write(model_to_json(pax_m))  # Save model
        
    # with open('dfsh_model_mx'+i+'.json', 'w') as fout:
    #     fout.write(model_to_json(dfsh_m))  # Save model
        
    # with open('drv_model_mx'+i+'.json', 'w') as fout:
    #     fout.write(model_to_json(drv_m))  # Save model

    #with open('serialized_model.json', 'r') as fin:
     #   m = model_from_json(fin.read())  # Load model


    # Create Future Dataframe PAX
    pfc_pred = pfc_m.make_future_dataframe(periods = testdays + forecast, freq = 'D')
    pax_pred = pax_m.make_future_dataframe(periods = testdays + forecast, freq = 'D')

    # Create Future Dataframe DRV
    dfsh_pred= dfsh_m.make_future_dataframe(periods = testdays + forecast, freq = 'D')
    drv_pred = drv_m.make_future_dataframe(periods = testdays + forecast, freq = 'D')

    # Merge Regressors pax
    pfc_pred = pfc_pred.merge(master_pfc_df, how = 'left', on = 'ds').merge(paychecks[paychecks.country_code == j], how = 'left', left_on = 'ds', right_on = 'date', suffixes = (None,'_x')).merge(date_pred, how = 'left', left_on = 'ds', right_on = 'date_value', suffixes = (None,'_x'))
    pfc_pred['paycheck_day'] = pfc_pred['paycheck_day_x']
    pfc_pred['Monday'] = pfc_pred['Monday_x']
    pfc_pred['Tuesday'] = pfc_pred['Tuesday_x']
    pfc_pred['Wednesday'] = pfc_pred['Wednesday_x']
    pfc_pred['Thursday'] = pfc_pred['Thursday_x']
    pfc_pred['Saturday'] = pfc_pred['Saturday_x']
    pfc_pred['Sunday'] = pfc_pred['Sunday_x']
    pfc_pred = pfc_pred.drop(columns =  ['paycheck_day_x','date','day_of_week','Monday_x','Tuesday_x','Wednesday_x', 'Thursday_x',  'Saturday_x', 'Sunday_x' ])
    pfc_pred['rank_ly'] = pfc_pred['rank'] +52
    pfc_pred = pfc_pred.merge(pfc_pred, how= 'left', left_on = ['rank', 'weekday'], right_on = ['rank_ly', 'weekday'], suffixes = (None, '_y'))
    
    pfc_pred = pfc_pred.drop(columns =  ['ds_y', 'y_y', 'paycheck_day_y','Monday_y', 'Tuesday_y', 'Wednesday_y', 'Thursday_y', 'Saturday_y',
        'Sunday_y', 'rank_y', 'rank_ly_y', 'rank_ly',	'country_code_y',	'date_value_y'])
    pfc_pred.pax_acq_burn_pct = np.where(pfc_pred['pax_acq_burn_pct'].isnull(),np.where(pfc_pred['pax_acq_burn_pct_y'].isnull(),0,pfc_pred['pax_acq_burn_pct_y']),pfc_pred['pax_acq_burn_pct'])
    
    #pfc_pred.drop_duplicates(inplace=True)

    pax_pred = pax_pred.merge(master_pax_df, how = 'left', on = 'ds').merge(paychecks[paychecks.country_code == j], how = 'left', left_on = 'ds', right_on = 'date', suffixes = (None,'_x')).merge(date_pred, how = 'left', left_on = 'ds', right_on = 'date_value', suffixes = (None,'_x'))

    pax_pred['paycheck_day'] = pax_pred['paycheck_day_x']
    pax_pred['Monday'] = pax_pred['Monday_x']
    pax_pred['Tuesday'] = pax_pred['Tuesday_x']
    pax_pred['Wednesday'] = pax_pred['Wednesday_x']
    pax_pred['Thursday'] = pax_pred['Thursday_x']
    pax_pred['Saturday'] = pax_pred['Saturday_x']
    pax_pred['Sunday'] = pax_pred['Sunday_x']
    pax_pred = pax_pred.drop(columns =  ['paycheck_day_x', 'date','date_value','day_of_week','Monday_x','Tuesday_x','Wednesday_x', 'Thursday_x',  'Saturday_x', 'Sunday_x' ])
    pax_pred['rank_ly'] = pax_pred['rank'] +52
    pax_pred = pax_pred.merge(pax_pred, how= 'left', left_on = ['rank', 'weekday'], right_on = ['rank_ly', 'weekday'], suffixes = (None, '_y'))
    pax_pred = pax_pred.drop(columns =  ['ds_y', 'y_y', 'paycheck_day_y','Monday_y', 'Tuesday_y', 'Wednesday_y', 'Thursday_y', 'Saturday_y', 'Sunday_y', 'rank_y', 'rank_ly_y','rank_ly','country_code_y'])
    pax_pred.pax_eng_burn_pct = np.where(pax_pred['pax_eng_burn_pct'].isnull(),np.where(pax_pred['pax_eng_burn_pct_y'].isnull(),0,pax_pred['pax_eng_burn_pct_y']),pax_pred['pax_eng_burn_pct'])

    #  Merge Regressors drv
    dfsh_pred = dfsh_pred.merge(master_dfsh_df, how = 'left', on = 'ds').merge(paychecks[paychecks.country_code == j], how = 'left', left_on = 'ds', right_on = 'date', suffixes = (None,'_x')).merge(date_pred, how = 'left', left_on = 'ds', right_on = 'date_value', suffixes = (None,'_x'))
    
    dfsh_pred['paycheck_day'] = dfsh_pred['paycheck_day_x']
    dfsh_pred['Monday'] = dfsh_pred['Monday_x']
    dfsh_pred['Tuesday'] = dfsh_pred['Tuesday_x']
    dfsh_pred['Wednesday'] = dfsh_pred['Wednesday_x']
    dfsh_pred['Thursday'] = dfsh_pred['Thursday_x']
    dfsh_pred['Saturday'] = dfsh_pred['Saturday_x']
    dfsh_pred['Sunday'] = dfsh_pred['Sunday_x']
    dfsh_pred = dfsh_pred.drop(columns =  ['paycheck_day_x', 'date','date_value','day_of_week','Monday_x','Tuesday_x','Wednesday_x', 'Thursday_x',  'Saturday_x', 'Sunday_x' ])
    dfsh_pred['rank_ly'] = dfsh_pred['rank'] +52
    dfsh_pred = dfsh_pred.merge(dfsh_pred, how= 'left', left_on = ['rank', 'weekday'], right_on = ['rank_ly', 'weekday'], suffixes = (None, '_y'))
    
    dfsh_pred = dfsh_pred.drop(columns =  ['ds_y', 'y_y', 'paycheck_day_y','Monday_y', 'Tuesday_y', 'Wednesday_y', 'Thursday_y', 'Saturday_y', 'Sunday_y', 'rank_y', 'rank_ly_y', 'rank_ly', 'country_code_y'])
    dfsh_pred.drv_acq_burn_pct = np.where(dfsh_pred['drv_acq_burn_pct'].isnull(),np.where(dfsh_pred['drv_acq_burn_pct_y'].isnull(),0,dfsh_pred['drv_acq_burn_pct_y']),dfsh_pred['drv_acq_burn_pct'])
    
    
    drv_pred = drv_pred.merge(master_drv_df, how = 'left', on = 'ds').merge(paychecks[paychecks.country_code == j], how = 'left', left_on = 'ds', right_on = 'date', suffixes = (None,'_x')).merge(date_pred, how = 'left', left_on = 'ds', right_on = 'date_value', suffixes = (None,'_x'))
    drv_pred['paycheck_day'] = drv_pred['paycheck_day_x']
    drv_pred['Monday'] = drv_pred['Monday_x']
    drv_pred['Tuesday'] = drv_pred['Tuesday_x']
    drv_pred['Wednesday'] = drv_pred['Wednesday_x']
    drv_pred['Thursday'] = drv_pred['Thursday_x']
    drv_pred['Saturday'] = drv_pred['Saturday_x']
    drv_pred['Sunday'] = drv_pred['Sunday_x']
    drv_pred = drv_pred.drop(columns =  ['paycheck_day_x','date_value','day_of_week','Monday_x','Tuesday_x','Wednesday_x', 'Thursday_x',  'Saturday_x', 'Sunday_x' ])
    drv_pred['rank_ly'] = drv_pred['rank'] +52
    drv_pred = drv_pred.merge(drv_pred, how= 'left', left_on = ['rank', 'weekday'], right_on = ['rank_ly', 'weekday'], suffixes = (None, '_y'))
    
    drv_pred = drv_pred.drop(columns =  ['ds_y', 'y_y', 'paycheck_day_y', 'Monday_y', 'Tuesday_y', 'Wednesday_y', 'Thursday_y', 'Saturday_y','Sunday_y', 'rank_y', 'rank_ly_y',  'rank_ly','country_code_y', 'date_y'])
    drv_pred.drv_eng_burn_pct = np.where(drv_pred['drv_eng_burn_pct'].isnull(),np.where(drv_pred['drv_eng_burn_pct_y'].isnull(),0,drv_pred['drv_eng_burn_pct_y']),drv_pred['drv_eng_burn_pct'])
    drv_pred.ipt_standardized = np.where(drv_pred['ipt_standardized'].isnull(),np.where(drv_pred['ipt_standardized_y'].isnull(),0,drv_pred['ipt_standardized_y']),drv_pred['ipt_standardized'])
    
    master_regressors = pfc_pred.merge(pax_pred, how = 'left', on = 'ds', suffixes = (None,'_x')).merge(dfsh_pred, how = 'left', on = 'ds', suffixes = (None,'_y')).merge(drv_pred, how = 'left', on = 'ds', suffixes = (None,'_z'))
    master_regressors = pd.DataFrame(master_regressors, columns =['ds','paycheck_day', 'pax_acq_burn_pct', 'pax_eng_burn_pct','drv_acq_burn_pct', 'drv_eng_burn_pct','ipt_standardized','ipt'])
    
    # Forecast PAX

    pfc_prediction = pfc_m.predict(pfc_pred)
    pfc_prediction = pfc_prediction.rename(columns = {'yhat':'pfc_prediction', 'yhat_lower': 'pfc_lower', 'yhat_upper': 'pfc_upper'}) 
    pax_prediction  = pax_m.predict(pax_pred)
    pax_prediction = pax_prediction.rename(columns = {'yhat':'pax_prediction', 'yhat_lower': 'pax_lower', 'yhat_upper': 'pax_upper'}) 

    # # Forecast DRV
    dfsh_prediction = dfsh_m.predict(dfsh_pred)
    dfsh_prediction = dfsh_prediction.rename(columns = {'yhat':'dfsh_prediction', 'yhat_lower': 'dfsh_lower', 'yhat_upper': 'dfsh_upper'}) 
    drv_prediction  = drv_m.predict(drv_pred)
    drv_prediction = drv_prediction.rename(columns = {'yhat':'drv_prediction', 'yhat_lower': 'drv_lower', 'yhat_upper': 'drv_upper'}) 

    predictions_df = pfc_prediction.merge(pax_prediction, how='left', on = 'ds',suffixes = (None, '_x')).merge(dfsh_prediction, how='left', on = 'ds', suffixes = (None, '_y')).merge(drv_prediction, how='left', on = 'ds', suffixes = (None, '_z'))
    predictions_df = pd.DataFrame(predictions_df, columns = ['ds', 'pfc_prediction', 'pax_prediction', 'dfsh_prediction' ,'drv_prediction']) #, 'pax_acq_burn_pct','pax_eng_burn_pct', 'drv_acq_burn_pct', 'drv_eng_burn_pct','ipt_standardized'])

    # Join Frequency with prediction
    market_df = frequency_df.merge(predictions_df, how = 'left', left_on = 'date_value', right_on = 'ds', suffixes = (None,'_x'))
    market_df = market_df.drop(columns = ['date_value'])
    market_df = market_df[(market_df.ds >= start_date)&(market_df.ds <= end_date)]

    market_df = pd.DataFrame(market_df, columns = [
        'year_calendar_week'
        ,'calendar_week'
        ,'ds'
        ,'city_id'
        ,'pfc_cpp_pred'
        ,'retained_cpp_pred'
        ,'dfsh_shpd_pred'
        ,'retained_drv_shpd_pred'
        ,'pfc_prediction'
        ,'pax_prediction'
        ,'dfsh_prediction' 
        ,'drv_prediction'])
    
    # Actuals dataframe
    rides_df = master.copy()
    rides_df['cr'] = rides_df.rides / rides_df.active_pax_calls
    rides_df = rides_df[(rides_df.city_id == i)][['current_stat_date','rides','active_pax_calls', 'active_drv_online_time','cr','pfc_cpp','pfc_pax_cnt','pfc_calls','retained_cpp','retained_pax_cnt','retained_calls','dfsh_shpd','dfsh_cnt', 'dfsh_online_time','retained_drv_shpd','retained_drv_cnt', 'retained_drv_online_time']]
    market_df = market_df.merge(master_regressors, how='left',on='ds', suffixes=(None,None))
    market_df = market_df.merge(rides_df, how='left',left_on = 'ds' ,right_on='current_stat_date', suffixes=(None,None))
    market_df.drop(columns=['current_stat_date'],inplace= True)
    market_df.city_id = market_df.city_id.astype('string')
    market_df.drop_duplicates(inplace= True)
    
    market_df = market_df.assign(
        predicted_calls = (market_df.pfc_cpp_pred*market_df.pfc_prediction) + (market_df.retained_cpp_pred*market_df.pax_prediction),
        predicted_tsh = (market_df.dfsh_shpd_pred*market_df.dfsh_prediction) + (market_df.retained_drv_shpd_pred*market_df.drv_prediction),
        predicted_dsr = ((market_df.pfc_cpp_pred*market_df.pfc_prediction) + (market_df.retained_cpp_pred*market_df.pax_prediction)) / ((market_df.dfsh_shpd_pred*market_df.dfsh_prediction) + (market_df.retained_drv_shpd_pred*market_df.drv_prediction)),
        predicted_pfc_calls = market_df.pfc_cpp_pred*market_df.pfc_prediction,
        predicted_retained_calls = market_df.retained_cpp_pred*market_df.pax_prediction,
        predicted_dfsh_tsh = market_df.dfsh_shpd_pred*market_df.dfsh_prediction,
        predicted_drv_tsh = market_df.retained_drv_shpd_pred*market_df.drv_prediction
    )
    
    market_df = pd.DataFrame(market_df, columns = [
        'year_calendar_week'
        ,'calendar_week'
        ,'ds'
        ,'city_id'
        ,'pfc_cpp_pred'
        ,'retained_cpp_pred'
        ,'dfsh_shpd_pred'
        ,'retained_drv_shpd_pred'
        ,'pfc_prediction'
        ,'pax_prediction'
        ,'dfsh_prediction' 
        ,'drv_prediction'
        ,'predicted_pfc_calls'
        ,'predicted_retained_calls'
        ,'predicted_dfsh_tsh'
        ,'predicted_drv_tsh'
        ,'predicted_calls'
        ,'predicted_tsh'
        ,'predicted_dsr'
        ,'pax_acq_burn_pct'
        ,'pax_eng_burn_pct'
        ,'drv_acq_burn_pct'
        ,'drv_eng_burn_pct'
        ,'ipt_standardized'
        ,'ipt'
        ,'pfc_pax_cnt'
        ,'pfc_calls'
        ,'pfc_cpp'
        ,'retained_pax_cnt'
        ,'retained_calls'
        ,'retained_cpp'
        ,'dfsh_cnt'
        ,'dfsh_online_time'
        ,'dfsh_shpd'
        ,'retained_drv_cnt'
        ,'retained_drv_online_time'
        ,'retained_drv_shpd' 
        ,'active_pax_calls'
        ,'active_drv_online_time'
        ,'cr'
        ,'rides'])

    market.append(market_df)

/var/folders/qq/jd7pmms11gv5jnzmr9cwqgk40000gn/T/ipykernel_9677/2574807851.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_holidays.date_value = pd.to_datetime(master_holidays.date_value, format='%Y-%m-%d')
17:38:04 - cmdstanpy - INFO - Chain [1] start processing
17:38:04 - cmdstanpy - INFO - Chain [1] done processing
17:38:06 - cmdstanpy - INFO - Chain [1] start processing
17:38:06 - cmdstanpy - INFO - Chain [1] start processing
17:38:06 - cmdstanpy - INFO - Chain [1] start processing
17:38:06 - cmdstanpy - INFO - Chain [1] done processing
17:38:06 - cmdstanpy - INFO - Chain [1] start processing
17:38:06 - cmdstanpy - INFO - Chain [1] start processing
17:38:06 - cmdstanpy - INFO - Chain [1] done processing
17:38:06 - cmdstanpy - INFO

17:39:56 - cmdstanpy - INFO - Chain [1] start processing
17:39:56 - cmdstanpy - INFO - Chain [1] done processing
17:39:56 - cmdstanpy - INFO - Chain [1] done processing
17:39:56 - cmdstanpy - ERROR - Chain [1] error: error during processing Stale NFS file handle
Optimization terminated abnormally. Falling back to Newton.
17:39:56 - cmdstanpy - INFO - Chain [1] start processing
17:39:56 - cmdstanpy - INFO - Chain [1] start processing
17:39:56 - cmdstanpy - INFO - Chain [1] start processing
17:39:56 - cmdstanpy - INFO - Chain [1] start processing
17:39:56 - cmdstanpy - INFO - Chain [1] start processing
17:39:56 - cmdstanpy - INFO - Chain [1] start processing
17:39:56 - cmdstanpy - INFO - Chain [1] done processing
17:39:56 - cmdstanpy - INFO - Chain [1] done processing
17:39:56 - cmdstanpy - INFO - Chain [1] done processing
17:39:56 - cmdstanpy - INFO - Chain [1] done processing
17:39:56 - cmdstanpy - INFO - Chain [1] start processing
17:39:56 - cmdstanpy - INFO - Chain [1] done processin

17:41:11 - cmdstanpy - INFO - Chain [1] start processing
17:41:11 - cmdstanpy - INFO - Chain [1] done processing
17:41:11 - cmdstanpy - INFO - Chain [1] done processing
17:41:11 - cmdstanpy - INFO - Chain [1] done processing
17:41:11 - cmdstanpy - ERROR - Chain [1] error: error during processing Stale NFS file handle
Optimization terminated abnormally. Falling back to Newton.
17:41:11 - cmdstanpy - INFO - Chain [1] start processing
17:41:11 - cmdstanpy - INFO - Chain [1] start processing
17:41:11 - cmdstanpy - INFO - Chain [1] start processing
17:41:11 - cmdstanpy - INFO - Chain [1] done processing
17:41:11 - cmdstanpy - INFO - Chain [1] done processing
17:41:11 - cmdstanpy - INFO - Chain [1] start processing
17:41:11 - cmdstanpy - INFO - Chain [1] start processing
17:41:11 - cmdstanpy - INFO - Chain [1] start processing
17:41:11 - cmdstanpy - INFO - Chain [1] start processing
17:41:11 - cmdstanpy - INFO - Chain [1] start processing
17:41:11 - cmdstanpy - INFO - Chain [1] done processi

17:41:28 - cmdstanpy - INFO - Chain [1] start processing
17:41:28 - cmdstanpy - INFO - Chain [1] done processing
17:41:30 - cmdstanpy - INFO - Chain [1] done processing
17:41:31 - cmdstanpy - INFO - Chain [1] done processing
17:41:31 - cmdstanpy - INFO - Chain [1] start processing
17:41:31 - cmdstanpy - INFO - Chain [1] done processing
17:41:33 - cmdstanpy - INFO - Chain [1] start processing
17:41:33 - cmdstanpy - INFO - Chain [1] start processing
17:41:33 - cmdstanpy - INFO - Chain [1] start processing
17:41:33 - cmdstanpy - INFO - Chain [1] start processing
17:41:33 - cmdstanpy - INFO - Chain [1] start processing
17:41:33 - cmdstanpy - INFO - Chain [1] start processing
17:41:33 - cmdstanpy - INFO - Chain [1] done processing
17:41:33 - cmdstanpy - INFO - Chain [1] start processing
17:41:33 - cmdstanpy - INFO - Chain [1] done processing
17:41:33 - cmdstanpy - INFO - Chain [1] done processing
17:41:33 - cmdstanpy - INFO - Chain [1] done processing
17:41:33 - cmdstanpy - INFO - Chain [1]

17:41:44 - cmdstanpy - INFO - Chain [1] done processing
17:41:44 - cmdstanpy - INFO - Chain [1] done processing
17:41:44 - cmdstanpy - INFO - Chain [1] done processing
17:41:44 - cmdstanpy - INFO - Chain [1] done processing
17:41:44 - cmdstanpy - INFO - Chain [1] start processing
17:41:44 - cmdstanpy - INFO - Chain [1] done processing
17:41:46 - cmdstanpy - INFO - Chain [1] start processing
17:41:46 - cmdstanpy - INFO - Chain [1] start processing
17:41:46 - cmdstanpy - INFO - Chain [1] start processing
17:41:46 - cmdstanpy - INFO - Chain [1] start processing
17:41:46 - cmdstanpy - INFO - Chain [1] start processing
17:41:46 - cmdstanpy - INFO - Chain [1] start processing
17:41:46 - cmdstanpy - INFO - Chain [1] done processing
17:41:46 - cmdstanpy - INFO - Chain [1] done processing
17:41:46 - cmdstanpy - INFO - Chain [1] done processing
17:41:46 - cmdstanpy - INFO - Chain [1] start processing
17:41:46 - cmdstanpy - INFO - Chain [1] start processing
17:41:46 - cmdstanpy - INFO - Chain [1]

...

17:48:47 - cmdstanpy - INFO - Chain [1] start processing
17:48:48 - cmdstanpy - INFO - Chain [1] done processing
17:48:49 - cmdstanpy - INFO - Chain [1] start processing
17:48:49 - cmdstanpy - INFO - Chain [1] start processing
17:48:49 - cmdstanpy - INFO - Chain [1] start processing
17:48:49 - cmdstanpy - INFO - Chain [1] start processing
17:48:49 - cmdstanpy - INFO - Chain [1] start processing
17:48:49 - cmdstanpy - INFO - Chain [1] start processing
17:48:49 - cmdstanpy - INFO - Chain [1] done processing
17:48:49 - cmdstanpy - INFO - Chain [1] start processing
17:48:49 - cmdstanpy - INFO - Chain [1] done processing
17:48:49 - cmdstanpy - INFO - Chain [1] done processing
17:48:49 - cmdstanpy - INFO - Chain [1] done processing
17:48:49 - cmdstanpy - INFO - Chain [1] done processing
17:48:49 - cmdstanpy - INFO - Chain [1] done processing
17:48:49 - cmdstanpy - INFO - Chain [1] start processing
17:48:50 - cmdstanpy - INFO - Chain [1] done processing
17:48:50 - cmdstanpy - INFO - Chain [1]

17:48:59 - cmdstanpy - INFO - Chain [1] done processing
17:49:00 - cmdstanpy - INFO - Chain [1] start processing
17:49:00 - cmdstanpy - INFO - Chain [1] start processing
17:49:00 - cmdstanpy - INFO - Chain [1] start processing
17:49:00 - cmdstanpy - INFO - Chain [1] start processing
17:49:00 - cmdstanpy - INFO - Chain [1] start processing
17:49:00 - cmdstanpy - INFO - Chain [1] start processing
17:49:00 - cmdstanpy - INFO - Chain [1] start processing
17:49:00 - cmdstanpy - INFO - Chain [1] done processing
17:49:00 - cmdstanpy - INFO - Chain [1] start processing
17:49:00 - cmdstanpy - INFO - Chain [1] done processing
17:49:00 - cmdstanpy - INFO - Chain [1] done processing
17:49:00 - cmdstanpy - INFO - Chain [1] done processing
17:49:00 - cmdstanpy - INFO - Chain [1] done processing
17:49:00 - cmdstanpy - INFO - Chain [1] done processing
17:49:00 - cmdstanpy - INFO - Chain [1] done processing
17:49:00 - cmdstanpy - INFO - Chain [1] done processing
17:49:01 - cmdstanpy - INFO - Chain [1] 

17:49:09 - cmdstanpy - INFO - Chain [1] start processing
17:49:10 - cmdstanpy - INFO - Chain [1] done processing
17:49:11 - cmdstanpy - INFO - Chain [1] start processing
17:49:11 - cmdstanpy - INFO - Chain [1] start processing
17:49:11 - cmdstanpy - INFO - Chain [1] start processing
17:49:11 - cmdstanpy - INFO - Chain [1] start processing
17:49:11 - cmdstanpy - INFO - Chain [1] start processing
17:49:11 - cmdstanpy - INFO - Chain [1] start processing
17:49:11 - cmdstanpy - INFO - Chain [1] done processing
17:49:11 - cmdstanpy - INFO - Chain [1] start processing
17:49:11 - cmdstanpy - INFO - Chain [1] start processing
17:49:11 - cmdstanpy - INFO - Chain [1] done processing
17:49:11 - cmdstanpy - INFO - Chain [1] done processing
17:49:11 - cmdstanpy - INFO - Chain [1] done processing
17:49:11 - cmdstanpy - INFO - Chain [1] done processing
17:49:11 - cmdstanpy - INFO - Chain [1] done processing
17:49:12 - cmdstanpy - INFO - Chain [1] done processing
17:49:12 - cmdstanpy - INFO - Chain [1]

17:49:20 - cmdstanpy - INFO - Chain [1] start processing
17:49:21 - cmdstanpy - INFO - Chain [1] done processing
17:49:22 - cmdstanpy - INFO - Chain [1] start processing
17:49:22 - cmdstanpy - INFO - Chain [1] start processing
17:49:22 - cmdstanpy - INFO - Chain [1] start processing
17:49:22 - cmdstanpy - INFO - Chain [1] done processing
17:49:23 - cmdstanpy - INFO - Chain [1] start processing
17:49:23 - cmdstanpy - INFO - Chain [1] start processing
17:49:23 - cmdstanpy - INFO - Chain [1] done processing
17:49:23 - cmdstanpy - INFO - Chain [1] done processing
17:49:23 - cmdstanpy - INFO - Chain [1] start processing
17:49:23 - cmdstanpy - INFO - Chain [1] start processing
17:49:23 - cmdstanpy - INFO - Chain [1] start processing
17:49:23 - cmdstanpy - INFO - Chain [1] done processing
17:49:23 - cmdstanpy - INFO - Chain [1] done processing
17:49:23 - cmdstanpy - INFO - Chain [1] done processing
17:49:23 - cmdstanpy - INFO - Chain [1] done processing
17:49:23 - cmdstanpy - INFO - Chain [1]

17:49:35 - cmdstanpy - INFO - Chain [1] start processing
17:49:35 - cmdstanpy - INFO - Chain [1] done processing
17:49:35 - cmdstanpy - INFO - Chain [1] start processing
17:49:35 - cmdstanpy - INFO - Chain [1] start processing
17:49:35 - cmdstanpy - INFO - Chain [1] done processing
17:49:35 - cmdstanpy - INFO - Chain [1] done processing
17:49:35 - cmdstanpy - INFO - Chain [1] start processing
17:49:35 - cmdstanpy - INFO - Chain [1] done processing
17:49:35 - cmdstanpy - ERROR - Chain [1] error: error during processing Stale NFS file handle
Optimization terminated abnormally. Falling back to Newton.
17:49:35 - cmdstanpy - INFO - Chain [1] start processing
17:49:35 - cmdstanpy - INFO - Chain [1] start processing
17:49:35 - cmdstanpy - INFO - Chain [1] start processing
17:49:36 - cmdstanpy - INFO - Chain [1] done processing
17:49:36 - cmdstanpy - INFO - Chain [1] done processing
17:49:36 - cmdstanpy - INFO - Chain [1] start processing
17:49:36 - cmdstanpy - INFO - Chain [1] start processi

In [ ]:
final_market_df = pd.concat(market)
filename = 'market_predictions_february_28d_daily'
folder = '28d_predictions'
predictions_folder = '2023 predictions'
final_market_df.to_csv('/Users/didi/Documents/Short Term Forecast/'+predictions_folder+'/'+folder+'/'+filename+'.csv', index = False)


final_market_df[final_market_df.ds >= "2022-07-16"].head(20)


In [ ]:
rides_df[rides_df.current_stat_date >= "2022-07-16"].head(20)

In [ ]:
# forecast_ssl = pd.concat(market)
# filename = 'market_predictions_february_28d'
# folder = 'Q1'
# predictions_folder = '2023 predictions'
# forecast_ssl.to_csv('/Users/didi/Documents/Short Term Forecast/'+predictions_folder+'/'+folder+'/'+filename.csv', index = False)

In [ ]:
# df = pd.read_csv('/Users/didi/Documents/Short Term Forecast/2023 Predictions/2023Q1/market_predictions2023test.csv')
# df = pd.DataFrame(df, columns = [
#        'year_calendar_week'
#        ,'calendar_week'
#        ,'ds'
#        ,'city_id'
#        ,'pfc_cpp_pred',
#        'retained_cpp_pred'
#        ,'dfsh_shpd_pred'
#        ,'retained_drv_shpd_pred'
#        ,'pfc_prediction'
#        ,'pax_prediction'
#        ,'dfsh_prediction'
#        ,'drv_prediction'
#        ,'predicted_calls'
#        ,'predicted_tsh'
#        ,'predicted_dsr'
#        ,'predicted_pfc_calls'
#        ,'predicted_retained_calls'
#        ,'predicted_dfsh_tsh'
#        ,'predicted_drv_tsh'
#        ,'rides'
#        ,'active_pax_calls'
#        ,'active_drv_online_time'
#        ,'cr'
#        ,'pfc_cpp'
#        ,'retained_cpp'
#        ,'dfsh_shpd'
#        ,'retained_drv_shpd'
#        ,'retained_pax_cnt'
#        ,'pfc_pax_cnt'
#        ,'dfsh_cnt'
#        ,'retained_drv_cnt']).drop_duplicates()

# df = df.groupby(['year_calendar_week','calendar_week','ds','city_id']).agg({
#     'pfc_cpp_pred':'mean',
#     'retained_cpp_pred':'mean',
#     'dfsh_shpd_pred':'mean',
#     'retained_drv_shpd_pred':'mean',
#     'pfc_prediction':'mean',
#     'pax_prediction':'mean',
#     'dfsh_prediction':'mean',
#     'drv_prediction':'mean',
#     'predicted_calls':'mean',
#     'predicted_tsh':'mean',
#     'predicted_dsr':'mean',
#     'predicted_pfc_calls':'mean',
#     'predicted_retained_calls':'mean',
#     'predicted_dfsh_tsh':'mean',
#     'predicted_drv_tsh':'mean',
#     'rides':'mean',
#     'active_pax_calls':'mean',
#     'active_drv_online_time':'mean',
#     'cr':'mean',
#     'pfc_cpp':'mean',
#     'retained_cpp':'mean',
#     'dfsh_shpd':'mean',
#     'retained_drv_shpd':'mean',
#     'retained_pax_cnt':'mean',
#     'pfc_pax_cnt':'mean',
#     'dfsh_cnt':'mean',
#     'retained_drv_cnt':'mean'
# }).reset_index()



In [ ]:
# filename = 'market_predictions2023Q1'
# folder = '2023Q1'
# predictions_folder = '2023 predictions'
# df.to_csv('/Users/didi/Documents/Short Term Forecast/'+predictions_folder+'/'+folder+'/'+filename+'SSLMature3'+'.csv', index = False)